# Temporal Tables (and GDPR)

Temporal, or System Versioned, Tables were introduced in SQL 2016, and are way of retaining a full change history on a table. When an existing record is changed, a record is added to a separate history table, showing how the record looked before the change, and the period for which that version of the record existed. In essence, it is a way of automatically producing a type 4 slowly changing dimension.

To produce a Temporal Table you first need a history table. This step can be skipped, and the history table  automatically generated with a system generated name, but I prefer for my tables to be descriptively named.

​The following script will generate my history table:

In [ ]:
if not exists (Select * from Information_Schema.Schemata where Schema_Name = 'History')
begin
	exec ('Create Schema History')
	;
end
go

Create table History.Applicant
(
	ApplicantID int not null,
	Forename varchar(50),
	Surname varchar(50),
	DateOfBirth date,
	ValidFrom datetime2 NOT NULL,  
	ValidTo datetime2 NOT NULL 
) with (Data_Compression = Page)
go


The important thing to note about the history table, is that it must have a period start and end date of type datetime2, and it must not have primary or foreign keys, check constraints, or identity fields. It is recommended that the history tables is clustered over the Valid From and To fields, and this is automatically applied on system generated tables.

Next we need to build a table to hold our current data table:

In [ ]:
if not exists (Select * from Information_Schema.Schemata where Schema_Name = 'CRM')
begin
	exec ('Create Schema CRM')
	;
end
go

Create table CRM.Applicant
(	
	ApplicantID int identity(1,1) primary key,
	Forename varchar(50),
	Surname varchar(50),
	DateOfBirth date,
	ValidFrom datetime2 GENERATED ALWAYS AS ROW START NOT NULL , 
	ValidTo datetime2 GENERATED ALWAYS AS ROW END NOT NULL ,    
	PERIOD FOR SYSTEM_TIME (ValidFrom,ValidTo) 
) with (Data_Compression = Page, SYSTEM_VERSIONING = ON (HISTORY_TABLE = History.Applicant))
go

This table must have:
* A primary key
* A pair of fields, of type datetime2, to hold the valid from and to periods. These fields will be automatically populated for new and modified records
* A PERIOD FOR SYSTEM_TIME statement, identifying your period fields
​* The SYSTEM_VERSIONING = ON statement, which enabled the versioning.

​Omitting the history table from the With clause results in a system generated table being created. 

​In SSMS's object explorer the history tables are not visible in their own right. Instead, they are displayed as a node under the main table.

![Temporal Table](./Images/temporal-history-table_orig.png "Temporal Table")

To demonstrate the way the temporal table works, I have run the following workload:

(Note: This query will run for 15 minutes to create a range of times. This can be modified by Waitfor statements, which specify a delay in the format Hour:Minute:Second.)

In [ ]:
insert CRM.Applicant (Forename, Surname, DateOfBirth)
values  ('Jon','Doe','1972-08-12'),
        ('Jane','Doe','1976-05-23')
go

Waitfor Delay '00:05:00'
go

Update  CRM.Applicant
Set     DateOfBirth = '1977-05-23'
where   ApplicantID = 2
go

Waitfor Delay '00:05:00'
go

Update  CRM.Applicant
Set     Surname = 'Dough'
where   ApplicantID = 2
go

Waitfor Delay '00:05:00'
go

Update  CRM.Applicant
Set     Forename = 'Jayne'
where   ApplicantID = 2
go

The current and history tables can be queried independently, with the history table showing every record version prior to the current version:

In [ ]:
​Select * from CRM.Applicant
Select * from History.Applicant

However; there is no need ever to query the history table at all, as you can interact with the history table transparently through the current table.

The following query will return data as it was at a particular point in time:

(Note: The valid time ranges in your data will reflect when you ran the script to populate your table; you will need to update the below query with an appropriate data and time)

In [ ]:
Select  * 
from    CRM.Applicant
FOR SYSTEM_TIME AS OF '2018-05-28 17:55:00'

![Results](./Images/temporal-2_orig.png "Results")

The next query returns all valid row versions between 2 points in time:

In [ ]:
​Select  * 
from    CRM.Applicant
FOR SYSTEM_TIME BETWEEN  '2018-05-28 17:57:00' AND '2018-05-28 18:05:00'

![Results](./Images/temporal-3_orig.png "Results")

This next query returns only those row versions that started and ended within a specified time period:

In [ ]:
​Select  * 
from    CRM.Applicant
FOR SYSTEM_TIME CONTAINED IN ('2018-05-28 17:57:00', '2018-05-28 18:05:00')

![Results](./Images/temporal-4_orig.png "Results")

And this final query returns all row versions, whether current or not:

In [ ]:
​Select  * 
from    CRM.Applicant
FOR SYSTEM_TIME ALL 

![Results](./Images/temporal-5_orig.png "Results")

## But how does this relate to GDPR?

Under GDPR regulations; you cannot retain data which permits identification of data subjects for any longer than is necessary, and so there will come a time when you have to delete this data. Lets give that a go...

In [ ]:
Delete CRM.Applicant where ApplicantID = 1
go

Select * from CRM.Applicant
Select * from History.Applicant
go

![Results](./Images/temporal-6_orig.png "Results")

The record has been deleted from the current table, but is now showing in the history table. So what if I delete the applicant from the history table?

In [ ]:
​Delete History.Applicant where ApplicantID = 1
go

![Results](./Images/temporal-7_orig.png "Results")

And herein lies the problem. The history is indelible; it cannot be directly modified or deleted. This is a major issue with regards to GDPR. The only way to remove data when it is no longer required is to disable system versioning, delete the records, and re-enable system versioning.

In [ ]:
Alter table CRM.Applicant
Set (SYSTEM_VERSIONING = OFF)
go

Delete History.Applicant where ApplicantID = 1
go

Alter table CRM.Applicant
Set (SYSTEM_VERSIONING = ON (HISTORY_TABLE = History.Applicant))
go

Select * from CRM.Applicant
Select * from History.Applicant
go

![Results](./Images/temporal-8_orig.png "Results")

But what do you do if you do not have a maintenance window, and need to have 5 9's up-time? If that is the case, then maybe temporal tables are not for you, and other methods change capture is more appropriate.

​​Happy SQLing!